In [24]:
using BenchmarkTools

In [146]:
module Ifc

using ComputedFieldTypes
using FunctionWrappers: FunctionWrapper

# Using an abstract parent type isn't necessary for this to work, 
# but I suspect it's something we'll often want to do
abstract type A end

struct B <: A
    x::Int
end

struct C <: A
    x::Float64
end

foo(a::A, y) = a.x + y

# This is the syntax I want to use eventually:
#=
@interface Foo{T} self::A begin
    foo(y::T)::T = self.x + y
end
=#
# The ComputedFieldTypes package also means that the return
# type of an interface method can be a computed type, like this:
#=
@interface Foo{T} self::A begin
    foo(y::T)::promote_op(+, Float64, T) = self.x + y
end
=#

# And the hypothetical `@interface` macro will produce something
# like the following:
@computed struct Foo{T}
    self::A
    foo::FunctionWrapper{Base.promote_op(+, Float64, T), Tuple{T}}

    Foo{T}(self) where {T} = new(self, y -> self.x + y)
end
foo(f::Foo{T}, y::T) where {T} = f.foo(y)

end

Ifc

In [147]:
# Demo
b = Ifc.B(1)
c = Ifc.C(2.0)
x_abstract = [b, c]

2-element Array{Ifc.A,1}:
 Ifc.B(1)  
 Ifc.C(2.0)

In [148]:
# The vector x_abstract has non-concrete type, so broadcast is slow:
y = zeros(2)
@benchmark $y .= Ifc.foo.($x_abstract, 2.0)

BenchmarkTools.Trial: 
  memory estimate:  80 bytes
  allocs estimate:  5
  --------------
  minimum time:     647.091 ns (0.00% GC)
  median time:      662.808 ns (0.00% GC)
  mean time:        732.005 ns (0.82% GC)
  maximum time:     14.280 μs (90.70% GC)
  --------------
  samples:          10000
  evals/sample:     164

In [154]:
# Instead, we can create an array of Foo interfaces:
x_interface = Ifc.Foo{Float64}.([b, c])

# The eltype of x_interface includes the (inferred) Float64
# return type:
eltype(x_interface)

Ifc.Foo{Float64,Float64}

In [155]:
# Calling the interface's methods is much faster:
y = zeros(2)
@benchmark $y .= Ifc.foo.($x_interface, 2.0)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     33.937 ns (0.00% GC)
  median time:      34.096 ns (0.00% GC)
  mean time:        35.244 ns (0.00% GC)
  maximum time:     163.404 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     993

In [156]:
# Note that there's still some unavoidable overhead,
# probably due to the lack of inlining, so converting
# x to a vector of floats will still be faster:
y = zeros(2)
x_float = [1.0, 2.0]
@benchmark $y .= (+).($x_float, 2.0)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     10.060 ns (0.00% GC)
  median time:      10.148 ns (0.00% GC)
  mean time:        10.273 ns (0.00% GC)
  maximum time:     20.380 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999